In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm

In [2]:
df=pd.read_csv('ObesityDataSet_raw_and_data_sinthetic.csv')
df

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.000000,1.620000,64.000000,yes,no,2.0,3.0,Sometimes,no,2.000000,no,0.000000,1.000000,no,Public_Transportation,Normal_Weight
1,Female,21.000000,1.520000,56.000000,yes,no,3.0,3.0,Sometimes,yes,3.000000,yes,3.000000,0.000000,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.000000,1.800000,77.000000,yes,no,2.0,3.0,Sometimes,no,2.000000,no,2.000000,1.000000,Frequently,Public_Transportation,Normal_Weight
3,Male,27.000000,1.800000,87.000000,no,no,3.0,3.0,Sometimes,no,2.000000,no,2.000000,0.000000,Frequently,Walking,Overweight_Level_I
4,Male,22.000000,1.780000,89.800000,no,no,2.0,1.0,Sometimes,no,2.000000,no,0.000000,0.000000,Sometimes,Public_Transportation,Overweight_Level_II
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2106,Female,20.976842,1.710730,131.408528,yes,yes,3.0,3.0,Sometimes,no,1.728139,no,1.676269,0.906247,Sometimes,Public_Transportation,Obesity_Type_III
2107,Female,21.982942,1.748584,133.742943,yes,yes,3.0,3.0,Sometimes,no,2.005130,no,1.341390,0.599270,Sometimes,Public_Transportation,Obesity_Type_III
2108,Female,22.524036,1.752206,133.689352,yes,yes,3.0,3.0,Sometimes,no,2.054193,no,1.414209,0.646288,Sometimes,Public_Transportation,Obesity_Type_III
2109,Female,24.361936,1.739450,133.346641,yes,yes,3.0,3.0,Sometimes,no,2.852339,no,1.139107,0.586035,Sometimes,Public_Transportation,Obesity_Type_III


In [13]:
macros_data=[]
bmr=[]
tdee=[]
for height, weight, gender, age, phy_activity in tqdm(zip(df['Height'], df['Weight'],df['Gender'], df['Age'],df['FAF'])):
    url = "https://www.calculator.net/macro-calculator.html"
    height_cm=height*100
    if gender=="Male":
        gender_par="m"
        bmr_1= 88.362 + (13.397 * weight) + (4.799 * height_cm) - (5.677 * age)
    else:
        gender_par="f"
        bmr_1= 447.593 + (9.247 * weight ) + (3.098 * height_cm) -(4.330 * age)

    bmr.append(bmr_1)
    
    if phy_activity== 0:
        td= 1.2
    elif phy_activity==1:
        td= 1.375
    elif phy_activity==2:
        td= 1.465
    elif phy_activity==3:
        td==1.55
    elif phy_activity==4:
        td= 1.725
    else:
        td== 1

    td_1=td*bmr_1
    tdee.append(td_1)

    params = {
        "ctype": "metric",
        "cage": age,
        "csex": gender_par,
        "cheightmeter": height_cm,
        "ckg": weight,
        "cactivity": td,
        "printit": 0,
        "x": 121,
        "y": 28
    }

    response = requests.get(url, params=params)

  # Create a Beautiful Soup object
    soup = BeautifulSoup(response.content, 'html.parser')

  # Find the recommended daily calorie intake
    macros = dict()

    labels = soup.find_all('td', {'class': 'arrow_box'})
    values = soup.find_all('td', {'class': 'result_box'})
    for label, value in zip(labels, values):
        temp_lab = label.div.text.strip()
        temp_val = value.text.replace('<', '').replace(':', '').strip().split(" ")
        macros[temp_lab+" ("+temp_val[1]+")"] = temp_val[0]
    # print(macros)
    macros_data.append(macros)


2111it [14:17,  2.46it/s]


In [16]:
macros_df = pd.DataFrame(macros_data)
macros_df

,Protein (grams/day),Carbs (grams/day),Fat (grams/day),Sugar (grams/day),Saturated Fat (grams/day),Food Energy (Calories/dayor)
0,101,222,47,44,19,"1,664"
1,91,199,42,40,17,"1,493"
2,159,349,74,70,30,"2,615"
3,167,364,78,73,31,"2,732"
4,139,305,65,61,26,"2,287"
...,...,...,...,...,...,...
2106,155,339,72,68,29,"2,541"
2107,158,345,74,69,29,"2,591"
2108,158,345,74,69,29,"2,590"
2109,156,342,73,68,29,"2,565"


In [18]:
final_df = df.join(macros_df)
final_df = final_df.dropna()
final_df['BMR']=bmr
final_df['TDEE']=tdee

ValueError: Length of values (2111) does not match length of index (1998)

In [8]:
final_df.to_csv("Obesity_with_Macros.xlsx")
print(len(bmr))